<a href="https://colab.research.google.com/github/angelv-salazar/escenarios-posnaturales/blob/main/sg2_ada_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrenamiento personalizado: StyleGan2-ADA

En este cuaderno, realizaremos el aprendizaje por transferencia con StyleGAN2 y conjuntos de datos personalizados.

Esto significa que no entrenaremos a la red GAN con nuestras imágenes desde cero (ya que toma alrededor de dos semanas) pero usaremos el modelo ya entrenado en las otras imágenes como punto de partida. Reducirá el tiempo de entrenamiento a aproximadamente 10 horas al omitir las primeras etapas donde la red neuronal aprende características de bajo nivel de imágenes que son casi las mismas para cualquier tipo de imágenes.

In [ ]:
#@title Montar Google Drive
#@markdown Monte Google Drive para cargar modelos previamente entrenados y guardar los resultados.

#@markdown Después de ejecutar esta celda, obtendrá el enlace. Haga click en el enlace, otorgue acceso a su Drive y copie el código de autenticación.

#@markdown Pegue el código en la entrada a continuación y presione Enter
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Instalación
#@markdown StyleGAN2-ADA será instalada en su Google Drive para acelerar el proceso de entrenamiento.

#@markdown Ejecute esta celda. Si ya ha instalado el repositorio, se omitirá el proceso de instalación y actualizará el directorio del repositorio. Si no lo ha instalado, instalará todos los archivos necesarios.
import os
if os.path.isdir("/content/drive/MyDrive/colab-sg2-ada-pytorch"):
    %cd "/content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
elif os.path.isdir("/content/drive/"):
    #install script
    %cd "/content/drive/MyDrive/"
    !mkdir colab-sg2-ada-pytorch
    %cd colab-sg2-ada-pytorch
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU -O /content/drive/MyDrive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch/pretrained/wikiart.pkl
else:
    !git clone https://github.com/dvschultz/stylegan2-ada-pytorch
    %cd stylegan2-ada-pytorch
    !mkdir downloads
    !mkdir datasets
    !mkdir pretrained
    %cd pretrained
    !gdown --id 1-5xZkD8ajXw1DdopTkH_rAoCsD72LhKU
    %cd ../

!pip install ninja opensimplex

%cd "/content/drive/My Drive/colab-sg2-ada-pytorch/stylegan2-ada-pytorch"
!git config --global user.name "test"
!git config --global user.email "test@test.com"
!git fetch origin
!git pull
!git stash
!git checkout origin/main -- train.py generate.py legacy.py closed_form_factorization.py flesh_digression.py apply_factor.py README.md calc_metrics.py training/stylegan2_multi.py training/training_loop.py util/utilgan.py

In [ ]:
#@title Preparación de Datos
#@markdown Directorio de entrada de Imágenes
input_dir = '' #@param {type: "string"}
#@markdown Ruta al archivo *zip* donde se almacenará el *Dataset* convertido. Usted debe crearlo la primera vez.
dataset_file = '' #@param {type: "string"}

if not dataset_file.endswith('.zip'):
  dataset_file += '.zip'

!python dataset_tool.py --source {input_dir} --dest {dataset_file}

In [ ]:
#@title Entrenamiento de Modelo Personalizado

#@markdown Ruta al archivo *zip* del *Dataset*
dataset = "" #@param {type: "string"}

#@markdown Para el aprendizaje por transferencia, configúrelo en **ffhq256**, **ffhq512** o **ffhq1024** de acuerdo con la resolución de sus imágenes.
#@markdown Si desea reanudar el proceso de capacitación, proporcione la ruta a su último archivo *.pkl*
resume_from = "" #@param {type: "string"}

#don't edit this unless you know what you're doing :)
!python train.py --outdir ./results --snap=1 --cfg='11gb-gpu' --data={dataset} --aug=noaug --mirror=False --mirrory=False --metrics=None --resume={resume_from}

### Mientras se está entrenando...
**Una vez que la celda anterior se esté ejecutando, ¡debería estar entrenando!**

¡No cierre esta pestaña! Colab debe estar abierto y en funcionamiento para poder seguir entrenando.

Cada 40 minutos más o menos debería añadirse una nueva línea a su salida, indicando que todavía está entrenando. Dependiendo de su configuración de `snapshot_count`, debería ver la carpeta de resultados (`/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/results`) en su carpeta de Google drive llenándose con ambas muestras (`fakesXXXXXx.jpg`) y *model weights* (pesos del modelo) (`network-snapshot-XXXXXX.pkl`). Vale la pena mirar las muestras mientras se entrena, pero no se preocupe demasiado por cada muestra individual.

Una vez que Colab se apaga, puede volver a conectarse en el alojamiento y volver a ejecutar cada celda de arriba a abajo. Asegúrese de actualizar la ruta `resume_from` para continuar entrenando desde el último modelo.